In [1]:
import pandas as pd
import numpy as np

In [94]:
df = pd.read_csv("dataport-export_gas_oct2015-mar2016.csv")
df.tail()

,localminute,dataid,meter_value
1584818,2016-03-31 23:59:14.336743-05,2129,201726
1584819,2016-03-31 23:59:17.427165-05,2945,161232
1584820,2016-03-31 23:59:35.370782-05,9729,138146
1584821,2016-03-31 23:59:47.816286-05,5129,166488
1584822,2016-03-31 23:59:58.92308-05,484,114174


In [139]:
time_df = df.copy()
time_df.localminute = time_df.localminute.transform(lambda x: x[0:-3])
time_df.localminute = time_df.localminute.transform(lambda x: pd.Timestamp(x).value)

In [117]:
#Each household should only have 1 meter, and thus 1 meter id per household. Finding num households
num_households = len(df["dataid"].unique())
display("The number of households is %s" %num_households)

'The number of households is 157'

In [4]:
#Fault occurs when cumulative readings go down
house_hold_ids = df["dataid"].unique()
faulty = []
for household in house_hold_ids:
    household_df = df[df["dataid"] == household]
    household_df.sort_values(by="localminute")
    if not household_df["meter_value"].is_monotonic:
        faulty.append(household)
len(faulty)

43

In [34]:
fault_timings = dict((key,[]) for key in faulty)
for faulty_meter in faulty:
    household_df = df[df["dataid"] == faulty_meter]
    x = household_df["meter_value"].to_numpy()
    xy = np.ediff1d(x)
    y = np.where(xy<0)
    y = [i+1 for i in y[0]]
    fault_timings[faulty_meter] = y

In [146]:
from scipy import interpolate
from scipy import stats
def get_correlation(a, b):
    timea,timeb = a.localminute, b.localminute
    start_time = max(timea.iloc[0],timeb.iloc[0])
    end_time = min(timea.iloc[-1],timeb.iloc[-1])
    time_series = np.linspace(start_time, end_time, max(len(a),len(b)))
    a_interp = interpolate.interp1d(timea,a.meter_value,fill_value="extrapolate")
    b_interp = interpolate.interp1d(timeb,b.meter_value,fill_value="extrapolate")
    a_new = a_interp(time_series)
    b_new = b_interp(time_series)
    return stats.pearsonr(a_new,b_new)

In [147]:
households = df["dataid"].unique()
correlation_df = pd.DataFrame(columns=households, index = households)
for i in range(num_households):
    base_household = time_df[time_df.dataid == households[i]]
    print(i)
    for j in range(i+1,num_households):
        comparison_household = time_df[time_df.dataid == households[j]]
        corr, _ = get_correlation(base_household,comparison_household)
        correlation_df.loc[households[i],households[j]] = corr
        correlation_df.loc[households[j],households[i]] = corr

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


/home/weiern/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/weiern/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/weiern/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/weiern/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/weiern/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConsta

21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138


/home/weiern/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/weiern/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/weiern/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/weiern/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/weiern/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConsta

139
140
141
142
143
144
145
146
147
148
149
150


/home/weiern/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/weiern/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


151
152
153
154
155
156


In [172]:
correlation_df = correlation_df.astype("float64")
top5_correlations = pd.DataFrame(index=households, columns=[1,2,3,4,5])
for household in households:
    top5_correlations.loc[household] = correlation_df.nlargest(5,household).index.to_list()

In [177]:
top5_correlations.to_csv("cor.csv")